In [1]:
import random
import networkx as nx
import itertools

In [2]:
def load_graph_from_file(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [3]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [4]:
def fitness(resolving_set,G,shortest_paths):
    unresolved_pairs = 0
    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova

In [5]:
def construct_solution(G,pheromones,heuristics,alpha,beta,shortest_paths):
    resolving_set = set()
    available_nodes = set(G.nodes)

    while available_nodes:
        # Računanje verovatnoće izbora svakog čvora
        probabilities = {node: (pheromones[node] ** alpha) * (heuristics[node] ** beta) for node in available_nodes}
        total = sum(probabilities.values())
        
        if total == 0:
            break
        
        # Normalizacija verovatnoća
        probabilities = {node: prob / total for node, prob in probabilities.items()}

        # Izbor čvora proporcionalno verovatnoći
        selected_node = random.choices(list(probabilities.keys()), weights=list(probabilities.values()))[0]
        resolving_set.add(selected_node)
        available_nodes.remove(selected_node)

        # Ako je rešenje dovoljno dobro, prekidamo ranije
        if fitness(resolving_set,G,shortest_paths) < 100:
            break

    return resolving_set

In [6]:
def update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths):
    # Isparavanje feromona
    print("Pre ažuriranja feromona:", pheromones)
    for node in pheromones:
        pheromones[node] *= (1 - evaporation)
    
    # Dodavanje novih feromona na čvorove iz najboljeg rešenja
    print("Posle isparavanja feromona:", pheromones)
    for node in best_solution:
        pheromones[node] += Q / fitness(best_solution,G,shortest_paths)

In [7]:
def ant_colony_optimization(G,num_iterations,num_ants,alpha,beta,evaporation,pheromones,heuristics,Q,shortest_paths):
    best_solution = None
    best_fitness = float("inf")

    for iteration in range(num_iterations):
        solutions = [construct_solution(G,pheromones,heuristics,alpha,beta,shortest_paths) for _ in range(num_ants)]
        sorted_solutions = sorted(solutions, key=lambda s: fitness(s, G,shortest_paths))        
        if fitness(sorted_solutions[0],G,shortest_paths) < best_fitness:
            best_solution = sorted_solutions[0]
            best_fitness = fitness(best_solution,G,shortest_paths)

        update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths)
        
        print(f"Iteracija {iteration}, najbolji skup {best_fitness},  {best_solution}")
    print("\nOptimalni rešavajući skup:", best_solution)
    return best_solution

In [8]:
# G = nx.Graph()
# G.add_edges_from([(1,2),(1,3),(1,5),(1,6),(2,4),(2,6),(3,5),(3,4),(4,5),(4,6),(5,6)])

# G = nx.petersen_graph()

G = load_graph_from_file('grafovi//gcol//gcol1.txt')

n = len(G.nodes)

# Parametri ACO algoritma
num_ants = 20     # Broj mrava po iteraciji
num_iterations = 50  # Broj iteracija
alpha = 1.0       # Uticaj feromona
beta = 2.0        # Uticaj heuristike (udaljenosti)
evaporation = 0.5  # Isparavanje feromona
Q = 100           # Konstanta za ažuriranje feromona

# Inicijalizacija feromona (svaki čvor dobija početnu vrednost)
pheromones = {node: 1.0 for node in G.nodes}

# Heuristička vrednost: obrnuta prosečna udaljenost od svih drugih čvorova
heuristics = {node: 1.0 / (sum(nx.shortest_path_length(G, node, j) for j in G.nodes if j != node) + 1) for node in G.nodes}


shortest_paths = dict(nx.all_pairs_shortest_path_length(G))  # Svi putevi između svih parova


rez = ant_colony_optimization(G,num_iterations,num_ants,alpha,beta,evaporation,pheromones,heuristics,Q,shortest_paths)


Pre ažuriranja feromona: {1: 1.0, 5: 1.0, 7: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0, 18: 1.0, 19: 1.0, 22: 1.0, 25: 1.0, 27: 1.0, 28: 1.0, 30: 1.0, 31: 1.0, 32: 1.0, 33: 1.0, 35: 1.0, 36: 1.0, 37: 1.0, 39: 1.0, 40: 1.0, 41: 1.0, 42: 1.0, 45: 1.0, 46: 1.0, 47: 1.0, 51: 1.0, 53: 1.0, 54: 1.0, 55: 1.0, 58: 1.0, 59: 1.0, 60: 1.0, 62: 1.0, 63: 1.0, 65: 1.0, 69: 1.0, 70: 1.0, 71: 1.0, 72: 1.0, 75: 1.0, 81: 1.0, 84: 1.0, 87: 1.0, 88: 1.0, 89: 1.0, 91: 1.0, 93: 1.0, 95: 1.0, 96: 1.0, 98: 1.0, 2: 1.0, 3: 1.0, 6: 1.0, 17: 1.0, 20: 1.0, 21: 1.0, 23: 1.0, 26: 1.0, 34: 1.0, 43: 1.0, 44: 1.0, 48: 1.0, 49: 1.0, 50: 1.0, 52: 1.0, 56: 1.0, 57: 1.0, 66: 1.0, 82: 1.0, 83: 1.0, 85: 1.0, 86: 1.0, 100: 1.0, 4: 1.0, 24: 1.0, 61: 1.0, 64: 1.0, 68: 1.0, 76: 1.0, 77: 1.0, 78: 1.0, 80: 1.0, 92: 1.0, 94: 1.0, 29: 1.0, 73: 1.0, 74: 1.0, 79: 1.0, 97: 1.0, 67: 1.0, 8: 1.0, 38: 1.0, 90: 1.0, 99: 1.0}
Posle isparavanja feromona: {1: 0.5, 5: 0.5, 7: 0.5, 9: 0.5, 10: 0.5, 11: 0.5, 12:

KeyboardInterrupt: 

In [ ]:
is_resolving_set(rez,G)

True